### Today we are going to perform the simple classification of the amazon reviews' sentiment.

### Please, download the dataset amazon_baby.csv.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
from sklearn.linear_model import LogisticRegression

def remove_punctuation(text):
    import string
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

baby_df = pd.read_csv('amazon_baby.csv')
baby_df.head()

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5


## Exercise 1 (data preparation)
a) Remove punctuation from reviews using the given function.   
b) Replace all missing (nan) revies with empty "" string.  
c) Drop all the entries with rating = 3, as they have neutral sentiment.   
d) Set all positive ($\geq$4) ratings to 1 and negative($\leq$2) to -1.

In [15]:
#a)
baby_df['review'] = baby_df['review'].fillna('').astype(str)

#short test:
baby_df["review"][4] == 'All of my kids have cried nonstop when I tried to ween them off their pacifier until I found Thumbuddy To Loves Binky Fairy Puppet  It is an easy way to work with your kids to allow them to understand where their pacifier is going and help them part from itThis is a must buy book and a great gift for expecting parents  You will save them soo many headachesThanks for this book  You all rock'


True

In [18]:
#b)
baby_df['review'] = baby_df['review'].fillna("")

#short test:
baby_df["review"][38] == baby_df["review"][38]

True

In [27]:
#c)
baby_df = baby_df[baby_df['rating'] != 3]

#short test:
sum(baby_df["rating"] == 3)

0

In [28]:
#d)
baby_df["rating"].replace(1,-1, inplace = True)
baby_df["rating"].replace(4,1, inplace = True)
baby_df["rating"].replace(5,1, inplace = True)
baby_df["rating"].replace(2,-1, inplace = True)


#short test:
sum(baby_df["rating"]**2 != 1)

/tmp/ipython-input-3892505790.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  baby_df["rating"].replace(1,-1, inplace = True)
/tmp/ipython-input-3892505790.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try usi

0

## CountVectorizer
In order to analyze strings, we need to assign them numerical values. We will use one of the simplest string representation, which transforms strings into the $n$ dimensional vectors. The number of dimensions will be the size of our dictionary, and then the values of the vector will represent the number of appereances of the given word in the sentence.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
reviews_train_example = ["We like apples",
                   "We hate oranges",
                   "I adore bananas",
                   "We like like apples and oranges",
                   "They dislike bananas"]

X_train_example = vectorizer.fit_transform(reviews_train_example)

print(vectorizer.get_feature_names())
print(X_train_example.todense())



['adore', 'and', 'apples', 'bananas', 'dislike', 'hate', 'like', 'oranges', 'they', 'we']
[[0 0 1 0 0 0 1 0 0 1]
 [0 0 0 0 0 1 0 1 0 1]
 [1 0 0 1 0 0 0 0 0 0]
 [0 1 1 0 0 0 2 1 0 1]
 [0 0 0 1 1 0 0 0 1 0]]


In [ ]:
reviews_test_example = ["They like bananas",
                   "We hate oranges bananas and apples",
                   "We love bananas"] #New word!

X_test_example = vectorizer.transform(reviews_test_example)

print(X_test_example.todense())

[[0 0 0 1 0 0 1 0 1 0]
 [0 1 1 1 0 1 0 1 0 1]
 [0 0 0 1 0 0 0 0 0 1]]


We should acknowledge few facts. Firstly, CountVectorizer does not take order into account. Secondly, it ignores one-letter words (this can be changed during initialization). Finally, for test values, CountVectorizer ignores words which are not in it's dictionary.

## Exercise 2
a) Split dataset into training and test sets.     
b) Transform reviews into vectors using CountVectorizer.

In [34]:
from sklearn.model_selection import train_test_split
#a)
X = baby_df['review']
y = baby_df['rating']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [37]:
#b)
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

print("Training shape:", X_train_vectorized.shape)
print("Test shape:", X_test_vectorized.shape)


Training shape: (49776, 64810)
Test shape: (12444, 64810)


## Exercise 3
a) Train LogisticRegression model on training data (reviews processed with CountVectorizer, ratings as they were).   
b) Print 10 most positive and 10 most negative words.

In [39]:
#a)
model = LogisticRegression()
model.fit(X_train_vectorized, y_train)

print("Training accuracy:", model.score(X_train_vectorized, y_train))
print("Test accuracy:", model.score(X_test_vectorized, y_test))


Training accuracy: 0.9659675345548056
Test accuracy: 0.9156219864995179


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [42]:
#b)
coefs = model.coef_[0]
words = np.array(vectorizer.get_feature_names_out())
sorted_indices = np.argsort(coefs)

print("10 most negative words:")
print(words[sorted_indices[:10]])

print("\n10 most positive words:")
print(words[sorted_indices[-10:]])

#hint: model.coef_, vectorizer.get_feature_names()

10 most negative words:
['worst' 'poorly' 'disappointing' 'frustrating' 'worthless' 'concept'
 'disappointed' 'dissapointed' 'terrible' 'poor']

10 most positive words:
['handy' 'complaint' 'pleased' 'fantastic' 'awesome' 'highly' 'perfect'
 'lifesaver' 'excellent' 'amazing']


## Exercise 4
a) Predict the sentiment of test data reviews.   
b) Predict the sentiment of test data reviews in terms of probability.   
c) Find five most positive and most negative reviews.   
d) Calculate the accuracy of predictions.

In [47]:
#a)
y_pred = model.predict(X_test_vectorized)

print(y_pred)


[1 1 1 ... 1 1 1]


In [49]:
#b)
probabilty = model.predict_proba(X_test_vectorized)

probabilty

#hint: model.predict_proba()

array([[5.71119931e-05, 9.99942888e-01],
       [1.60951870e-04, 9.99839048e-01],
       [4.75752835e-03, 9.95242472e-01],
       ...,
       [1.09002779e-03, 9.98909972e-01],
       [2.13959124e-02, 9.78604088e-01],
       [1.35136073e-04, 9.99864864e-01]])

In [50]:
#c)
print("most positive: ", test.iloc[probabilty.T[1].argmax()]["review"])

print("\n most negative: ", test.iloc[probabilty.T[0].argmax()]["review"])

#hint: use the results of b)

most positive:  We bought this for our daughter for a Christmas present we just received it in the mail today We put it together then put her in it just to see how well she liked it Well I think she is going to be getting an early present She loves it The second we sat her down in it she started going crazy She loves all of the toys and loves the bounce I was really concerned with how much it would actually bounce and it definitely went above and beyond my expectations She is 6 months old and my only regret is that we didnt get her one sooner She stayed in it for about an hour and a half then I went ahead and took her out because I figured her legs were probably getting tired She started crying when I took her out she wanted back in I sat down with her in her rocking char and poor baby just fell right asleep This is the best toy ever invented She isnt normally very interested in toys but loves this one Every baby needs one of these

 most negative:  This was exactly what we were lookin

In [53]:
#d)
accuracy = model.score(X_test_vectorized, y_test)
print("Accuracy on test set:", accuracy)


Accuracy on test set: 0.9156219864995179


## Exercise 5
In this exercise we will limit the dictionary of CountVectorizer to the set of significant words, defined below.


a) Redo exercises 2-5 using limited dictionary.   
b) Check the impact of all the words from the dictionary.   
c) Compare accuracy of predictions and the time of evaluation.

In [ ]:
significant_words = ['love','great','easy','old','little','perfect','loves','well','able','car','broke','less','even','waste','disappointed','work','product','money','would','return']

In [ ]:
#a)


In [ ]:
#b)


In [ ]:
#c)

#hint: %time, %timeit